# Trend Following Trading System - Quick Start

This notebook demonstrates the trend-following trading system based on Tom Basso's principles.

**Strategy Overview:**
- Trade 30 sector ETFs long and short
- Keltner Channel breakout entries
- ATR-based position sizing (2% volatility target)
- VaR-based risk management (95% confidence, 20% max)
- Trailing stops activated after 1×ATR profit

## 1. Setup & Configuration

In [ ]:
# Standard imports
import sys
from pathlib import Path
from datetime import date, timedelta

# Add project root to path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Project imports
from src.utils.config import load_config
from src.data.database import Database
from src.data.fetcher import DataFetcher
from src.backtest.engine import BacktestEngine
from src.reporting.console import print_metrics, print_trade_summary
from src.reporting.charts import create_backtest_report, plot_equity_curve, plot_drawdown

import matplotlib.pyplot as plt
%matplotlib inline

print("Imports successful!")

In [ ]:
# Load configuration
config = load_config(project_root / "config.yaml")

print(f"Starting Capital: ${config.portfolio.starting_capital:,}")
print(f"ETF Universe: {len(config.data.etf_symbols)} symbols")
print(f"Keltner Period: {config.strategy.keltner.ema_period}")
print(f"ATR Multiplier: {config.strategy.keltner.atr_multiplier}")
print(f"Max VaR: {config.risk_management.var.max_var_pct}%")

## 2. Fetch Historical Data

In [ ]:
# Initialize database and fetcher
db_path = project_root / config.data.database_path
db_path.parent.mkdir(parents=True, exist_ok=True)

db = Database(str(db_path))
fetcher = DataFetcher(db)

print(f"Database: {db_path}")

In [ ]:
# Fetch data for all ETFs (this may take a few minutes on first run)
symbols = config.data.etf_symbols + [config.data.benchmark_symbol]

# Calculate start date (5 years ago)
start_date = date.today() - timedelta(days=5*365)

print(f"Fetching data for {len(symbols)} symbols from {start_date}...")
results = fetcher.fetch_multiple(symbols, start_date=start_date)

success = sum(1 for r in results.values() if r > 0)
print(f"Successfully updated {success}/{len(symbols)} symbols")

In [ ]:
# Load data from database
data = {}
for symbol in config.data.etf_symbols:
    df = db.get_price_data(symbol)
    if df is not None and len(df) > 100:
        data[symbol] = df

# Load benchmark
benchmark_data = db.get_price_data(config.data.benchmark_symbol)

print(f"Loaded {len(data)} symbols with sufficient history")
if data:
    sample = list(data.values())[0]
    print(f"Date range: {sample.index[0].date()} to {sample.index[-1].date()}")

## 3. Run Backtest

In [ ]:
# Initialize backtest engine
engine = BacktestEngine(config)

print("Backtest Engine Configuration:")
print(f"  - Keltner EMA: {config.strategy.keltner.ema_period}")
print(f"  - ATR Period: {config.strategy.keltner.atr_period}")
print(f"  - ATR Multiplier: {config.strategy.keltner.atr_multiplier}")
print(f"  - Initial Stop ATR: {config.strategy.stops.initial_atr_multiple}")
print(f"  - Trailing Stop ATR: {config.strategy.stops.trailing_atr_multiple}")

In [ ]:
# Run backtest
result = engine.run(
    data=data,
    start_date=date(2020, 1, 1),  # Adjust as needed
    end_date=None,  # Run to latest date
    benchmark_data=benchmark_data,
)

print(f"Backtest complete!")
print(f"Total trades: {len(result.trades)}")

## 4. Performance Results

In [ ]:
# Print performance metrics
print_metrics(result.metrics)

In [ ]:
# Print trade summary (first 20 trades)
print_trade_summary(result.trades, max_trades=20)

## 5. Visualizations

In [ ]:
# Create benchmark equity curve for comparison
benchmark_equity = None
if benchmark_data is not None and len(benchmark_data) > 0 and len(result.equity_curve) > 0:
    # Align benchmark to backtest period
    start = result.equity_curve.index[0]
    end = result.equity_curve.index[-1]
    bench = benchmark_data.loc[start:end, "close"]
    if len(bench) > 0:
        # Normalize to same starting value
        benchmark_equity = bench / bench.iloc[0] * result.equity_curve.iloc[0]

# Plot equity curve
fig = plot_equity_curve(
    result.equity_curve,
    benchmark_curve=benchmark_equity,
    title="Portfolio vs Benchmark (SPY)",
)
plt.show()

In [ ]:
# Plot drawdown
fig = plot_drawdown(result.equity_curve, title="Portfolio Drawdown")
plt.show()

In [ ]:
# Create comprehensive report
fig = create_backtest_report(
    metrics=result.metrics,
    equity_curve=result.equity_curve,
    trades=result.trades,
    benchmark_curve=benchmark_equity,
    title="Trend Following Strategy - Full Report",
)
plt.show()

## 6. Trade Analysis

In [ ]:
# Analyze trades by symbol
import pandas as pd

if result.trades:
    trade_df = pd.DataFrame([
        {
            "symbol": t.symbol,
            "side": t.side.name,
            "pnl": t.pnl,
            "pnl_pct": t.pnl_pct,
            "holding_days": t.holding_days,
            "exit_reason": t.exit_reason,
        }
        for t in result.trades
    ])
    
    # Summary by symbol
    symbol_summary = trade_df.groupby("symbol").agg({
        "pnl": ["count", "sum", "mean"],
        "holding_days": "mean",
    }).round(2)
    symbol_summary.columns = ["trades", "total_pnl", "avg_pnl", "avg_days"]
    symbol_summary = symbol_summary.sort_values("total_pnl", ascending=False)
    
    print("Top 10 Symbols by P&L:")
    print(symbol_summary.head(10))
    print("\nBottom 10 Symbols by P&L:")
    print(symbol_summary.tail(10))
else:
    print("No trades to analyze.")

In [ ]:
# Exit reason analysis
if result.trades:
    exit_summary = trade_df.groupby("exit_reason").agg({
        "pnl": ["count", "sum", "mean"],
    }).round(2)
    exit_summary.columns = ["count", "total_pnl", "avg_pnl"]
    print("Trades by Exit Reason:")
    print(exit_summary)

In [ ]:
# Long vs Short performance
if result.trades:
    side_summary = trade_df.groupby("side").agg({
        "pnl": ["count", "sum", "mean"],
        "pnl_pct": "mean",
    }).round(2)
    side_summary.columns = ["count", "total_pnl", "avg_pnl", "avg_pct"]
    print("Long vs Short Performance:")
    print(side_summary)

## 7. Save Results

In [ ]:
# Save equity curve to CSV
output_dir = project_root / "output"
output_dir.mkdir(exist_ok=True)

if len(result.equity_curve) > 0:
    result.equity_curve.to_csv(output_dir / "equity_curve.csv")
    print(f"Equity curve saved to {output_dir / 'equity_curve.csv'}")

    # Save trades to CSV
    if result.trades:
        trade_df.to_csv(output_dir / "trades.csv", index=False)
        print(f"Trades saved to {output_dir / 'trades.csv'}")
else:
    print("No results to save.")

In [ ]:
# Save report figure
if len(result.equity_curve) > 0:
    fig = create_backtest_report(
        metrics=result.metrics,
        equity_curve=result.equity_curve,
        trades=result.trades,
        benchmark_curve=benchmark_equity,
        title="Trend Following Strategy Report",
        save_path=output_dir / "backtest_report.png",
    )
    plt.close(fig)
    print(f"Report saved to {output_dir / 'backtest_report.png'}")

## 8. Next Steps

1. **Parameter Optimization**: Try different Keltner periods, ATR multipliers, and stop levels
2. **Walk-Forward Testing**: Use rolling windows to validate out-of-sample performance
3. **Live Trading**: Connect to Interactive Brokers using the execution layer
4. **Risk Analysis**: Examine correlation with benchmark during different market regimes

In [ ]:
# Clean up
db.close()
print("Database connection closed.")